In [ ]:

import pandas as pd
from sklearn import svm
from sklearn.decomposition import PCA
from skimage.io import imshow, show
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from scipy import ndimage

from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from PreProcessing.Processing import Processing
from FeaturesExtraction.Features import Features


# Processing The Data

In [ ]:

Processing_obj=Processing('dataset2_large')
data_train = []
labels = []

rotation_angles = [10, -10, 20, -20]  # Define a list of angles for data augmentation

data_trian,labels=Processing_obj.ReadAndProcess(rotation_angles)

print(f'Number of images loaded: {len(data_train)}')

# Get 20 random indices from data_train
random_indices = np.random.choice(len(data_train), size=20, replace=False)

# Plot the 20 random grayscale images
fig, axes = plt.subplots(nrows=4, ncols=5, figsize=(12, 10))

Processing_obj.ShowSomeImages(axes,data_train,labels,random_indices)

# Features Extraction

In [ ]:
Features_obj=Features()

hog_features = [Features_obj.extract_hog_features(image) for image in data_train]
print(f'HOG feature vector size: {hog_features[0].shape}')

lbp_features = [Features_obj.extract_lbp_features(image) for image in data_train]

# Flatten LBP features
flattened_lbp_features = [Features_obj.lbp_feature.flatten() for lbp_feature in lbp_features]
print(f'Length of flattened LBP features: {len(flattened_lbp_features)}')

# Concatenate HOG, LBP, and Gabor features
combined_features = np.hstack((hog_features, flattened_lbp_features))

print(f'Combined HOG, LBP features shape: {combined_features.shape}')


In [ ]:
# Apply PCA for dimensionality reduction
pca = PCA(n_components=0.7)
reduced_features = pca.fit_transform(combined_features)
print(f'Reduced features shape: {reduced_features.shape}')


In [ ]:
classifiers = [
    KNeighborsClassifier(n_neighbors=7),
    # DecisionTreeClassifier(),
    # RandomForestClassifier(),
    # AdaBoostClassifier(),
    # GradientBoostingClassifier(),
    # LogisticRegression(),
    MLPClassifier(),
    SVC(kernel='linear', C=1)
]

# Initialize a dictionary to store the classifier names and their accuracies
classifier_accuracies = {clf.__class__.__name__: [] for clf in classifiers}

# Define the number of iterations
num_iterations = 15

for i in range(num_iterations):
    X_train, X_test, y_train, y_test = train_test_split(reduced_features, labels, test_size=0.1)

    # Scale the data
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    for clf in classifiers:
        clf_name = clf.__class__.__name__
        clf.fit(X_train, y_train)
        accuracy = clf.score(X_test, y_test)
        classifier_accuracies[clf_name].append(accuracy)

# Print all accuracies for each classifier and the average accuracy
for clf_name, accuracies in classifier_accuracies.items():
    print(f'{clf_name} accuracies: {accuracies}')
    avg_accuracy = np.mean(accuracies)
    print(f'{clf_name} average accuracy: {avg_accuracy}\n')

# Print the classifier with the best average accuracy
best_clf_name = max(classifier_accuracies, key=lambda x: np.mean(classifier_accuracies[x]))
print(f'Best classifier: {best_clf_name} with average accuracy {np.mean(classifier_accuracies[best_clf_name])}')